In [2]:
path = '/notebook/kaggle/'
PROJ_HOME_DIR = path + 'data/ultrasound/'
DATA_HOME_DIR = PROJ_HOME_DIR # + 'sample/'

results_path = DATA_HOME_DIR + 'results/'
test_path = DATA_HOME_DIR + 'test/'
valid_path = DATA_HOME_DIR + 'valid/'
train_path = DATA_HOME_DIR + 'train/'

In [3]:
'''
#!kg config -g -u `username` -p `password` 

##dogs-vs-cats-redux-kernels-edition
##state-farm-distracted-driver-detection
##titanic
!kg config -c 'ultrasound-nerve-segmentation'

%cd $DATA_HOME_DIR
!kg download

%run -i 'data.py'
'''

"\n#!kg config -g -u `username` -p `password` \n\n##dogs-vs-cats-redux-kernels-edition\n##state-farm-distracted-driver-detection\n##titanic\n!kg config -c 'ultrasound-nerve-segmentation'\n\n%cd $DATA_HOME_DIR\n!kg download\n\n%run -i 'data.py'\n"

In [4]:
!pip install scikit-image

    100% |################################| 35.2MB 32kB/s  eta 0:00:01
    100% |################################| 5.8MB 199kB/s eta 0:00:01
    100% |################################| 5.7MB 199kB/s eta 0:00:01
    100% |################################| 81kB 6.3MB/s eta 0:00:01
  Running setup.py bdist_wheel for olefile ... done
  Stored in directory: /root/.cache/pip/wheels/20/58/49/cc7bd00345397059149a10b0259ef38b867935ea2ecff99a9b
Successfully built olefile


In [1]:
!echo $CUDA_VISIBLE_DEVICES

0


In [5]:
from __future__ import print_function

import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K

import sys
sys.path.append('/notebook/kaggle/data/ultrasound')
from data import load_train_data, load_test_data

K.set_image_data_format('channels_last')  # TF dimension ordering in this code

img_rows = 96
img_cols = 96

smooth = 1.

Using TensorFlow backend.


In [6]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

Define the model

In [7]:
def get_unet():
    inputs = Input((img_rows, img_cols, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model

In [8]:
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

In [9]:
# Load train data
imgs_train, imgs_mask_train = load_train_data()

imgs_train = preprocess(imgs_train)
imgs_mask_train = preprocess(imgs_mask_train)

imgs_train = imgs_train.astype('float32')
mean = np.mean(imgs_train)  # Mean for data centering
std = np.std(imgs_train)  # std for data normalization

imgs_train -= mean
imgs_train /= std

imgs_mask_train = imgs_mask_train.astype('float32')
imgs_mask_train /= 255.  # scale masks to [0, 1]

/usr/local/lib/python2.7/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Create and compile model

In [10]:
model = get_unet()
model_checkpoint = ModelCheckpoint(results_path+'weights.h5', monitor='val_loss', save_best_only=True)

Train model

In [11]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4731181354816385218
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 12041302836
locality {
  bus_id: 1
}
incarnation: 7078044972186964789
physical_device_desc: "device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0"
]


In [14]:
model.fit(imgs_train, imgs_mask_train, batch_size=32, epochs=20, verbose=1, shuffle=True,
          validation_split=0.2,
          callbacks=[model_checkpoint])

Train on 4508 samples, validate on 1127 samples
Epoch 1/20
4508/4508 [==============================] - 18s - loss: -2.9137e-04 - dice_coef: 2.9137e-04 - val_loss: -3.1876e-04 - val_dice_coef: 3.1876e-04
Epoch 2/20
4508/4508 [==============================] - 17s - loss: -2.9174e-04 - dice_coef: 2.9174e-04 - val_loss: -3.1876e-04 - val_dice_coef: 3.1876e-04
Epoch 3/20
4508/4508 [==============================] - 17s - loss: -2.9315e-04 - dice_coef: 2.9315e-04 - val_loss: -3.1876e-04 - val_dice_coef: 3.1876e-04
Epoch 4/20
4508/4508 [==============================] - 17s - loss: -2.9452e-04 - dice_coef: 2.9452e-04 - val_loss: -3.1876e-04 - val_dice_coef: 3.1876e-04
Epoch 5/20
4508/4508 [==============================] - 17s - loss: -2.8893e-04 - dice_coef: 2.8893e-04 - val_loss: -3.1876e-04 - val_dice_coef: 3.1876e-04
Epoch 6/20
4508/4508 [==============================] - 17s - loss: -2.9146e-04 - dice_coef: 2.9146e-04 - val_loss: -3.1876e-04 - val_dice_coef: 3.1876e-04
Epoch 7/20
4508/

Make predictions on test data

In [15]:
# Load and preprocess test data
imgs_test, imgs_id_test = load_test_data()
imgs_test = preprocess(imgs_test)

imgs_test = imgs_test.astype('float32')
imgs_test -= mean
imgs_test /= std

In [17]:
# Load saved weights
model.load_weights(results_path+'weights.h5')

# Predict masks using test data
imgs_mask_test = model.predict(imgs_test, verbose=1)
np.save('imgs_mask_test.npy', imgs_mask_test)

5472/5508 [============================>.] - ETA: 0s

In [18]:
# Save predicted masks to file
pred_dir = results_path+'preds'
if not os.path.exists(pred_dir):
    os.mkdir(pred_dir)
for image, image_id in zip(imgs_mask_test, imgs_id_test):
    image = (image[:, :, 0] * 255.).astype(np.uint8)
    imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/164_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1953_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2716_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3473_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/106_pred.png is a low contrast image
  warn('%s is a low co

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3456_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2802_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/4261_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1893_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/212_pred.png is a low contrast image
  warn('%s is a low c

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/5440_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/5162_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/4007_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3060_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/918_pred.png is a low contrast image
  warn('%s is a low c

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3220_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1357_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/5321_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1067_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1903_pred.png is a low contrast image
  warn('%s is a low 

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3453_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/5192_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/405_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3969_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2773_pred.png is a low contrast image
  warn('%s is a low c

...

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1146_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2162_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3309_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/5121_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3712_pred.png is a low contrast image
  warn('%s is a low 

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/4711_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3628_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2097_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1182_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/408_pred.png is a low contrast image
  warn('%s is a low c

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1851_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1772_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/487_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1072_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/409_pred.png is a low contrast image
  warn('%s is a low co

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/4576_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2529_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2532_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1586_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2625_pred.png is a low contrast image
  warn('%s is a low 

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2943_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/4905_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/5261_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/4299_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2222_pred.png is a low contrast image
  warn('%s is a low 